In [ ]:
#importing all necessary packages
import pandas as pd
import geopandas as gpd
import json
from bokeh.io import output_notebook, show, output_file, save, export_png
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

In [ ]:
#saving map shapefile
india = gpd.read_file('shapefile')
india.head()

In [ ]:
#import datafile
datafile = 'data.csv'
df = pd.read_csv(datafile, names = ['ST_NM', 'state_pos', 'migrant_pos'], skiprows = 1)
df.head()

In [ ]:
#combine shapefile and prevalence data
merged = india.merge(df, left_on = 'ST_NM', right_on = 'ST_NM')

In [ ]:
#convert to GeoJSON
merged_json = json.loads(merged.to_json())
json_data = json.dumps(merged_json)

In [ ]:
#set up map info
geosource = GeoJSONDataSource(geojson = json_data)
palette = brewer['OrRd'][8]
palette = palette[::-1]
color_mapper = LinearColorMapper(palette = palette, low = 0, high = .16, nan_color = '#d9d9d9')
tick_labels = {'0': '0%', '0.02': '2%', '0.04': '4%', '0.06': '6%', '0.08':'8%', '0.1': '10%', 
               '0.12':'12%', '0.14': '14%', '0.16':'>16%'}
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

In [ ]:
#create and save figure
migrant = figure(title = 'Returning worker positives rates (no data in gray regions)', plot_height = 800 , plot_width = 650, toolbar_location = None)
migrant.xgrid.grid_line_color = None
migrant.ygrid.grid_line_color = None
migrant.patches('xs','ys', source = geosource,fill_color = {'field' :'migrant_pos', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

migrant.add_layout(color_bar, 'below')
output_notebook()
show(migrant)
export_png(migrant, filename = 'migrant.png')

In [ ]:
#repeat for state reported data
state = figure(title = 'State-reported positives rates (no data in gray regions)', plot_height = 800 , plot_width = 650, toolbar_location = None)
state.xgrid.grid_line_color = None
state.ygrid.grid_line_color = None
state.patches('xs','ys', source = geosource,fill_color = {'field' :'state_pos', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

state.add_layout(color_bar, 'below')
output_notebook()
show(state)
export_png(state, filename = 'state.png')